# Class

In [1]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np

class RegressionMetrics:
    """
    A class for computing and printing regression evaluation metrics.

    Args:
        pipeline: The regression pipeline model.
        X_test: The test features.
        y_test: The test labels.
        X_val: The validation features.
        y_val: The validation labels.
        style: Flag indicating whether to apply styling to the output. Default is False.
    """

    def __init__(self, pipeline, X_test, y_test, X_val, y_val, style=False):
        self.pipeline = pipeline
        self.X_test = X_test
        self.y_test = y_test
        self.X_val = X_val
        self.y_val = y_val
        self.y_pred = None
        self.style = style

    def mean_absolute_error(self):
        """
        Calculates the mean absolute error (MAE).

        Returns:
            The mean absolute error value.
        """
        return mean_absolute_error(self.y_val, self.y_pred)

    def mean_absolute_percentage_error(self):
        """
        Calculates the mean absolute percentage error (MAPE).

        Returns:
            The mean absolute percentage error value.
        """
        return mean_absolute_percentage_error(self.y_val, self.y_pred)

    def mean_squared_error(self):
        """
        Calculates the mean squared error (MSE).

        Returns:
            The mean squared error value.
        """
        return mean_squared_error(self.y_val, self.y_pred)

    def mean_root_mean_squared_error(self):
        """
        Calculates the root mean squared error (RMSE).

        Returns:
            The root mean squared error value.
        """
        return mean_squared_error(self.y_val, self.y_pred, squared=False)

    def r2_test(self):
        """
        Calculates the R-squared score for the test set.

        Returns:
            The R-squared score for the test set.
        """
        return self.pipeline.score(self.X_test, self.y_test)

    def r2_val(self):
        """
        Calculates the R-squared score for the validation set.

        Returns:
            The R-squared score for the validation set.
        """
        return r2_score(self.y_val, self.y_pred)

    def aic(self):
        """
        Calculates the Akaike Information Criterion (AIC).

        Returns:
            The Akaike Information Criterion value.
        """
        y_pred = self.pipeline.predict(self.X_val)
        n_params = len(self.pipeline.named_steps)
        n = len(self.y_val)
        mse = mean_squared_error(self.y_val, y_pred)
        aic = 2 * n_params - 2 * np.log(mse) + n_params * np.log(n)
        return aic

    def bic(self):
        """
        Calculates the Bayesian Information Criterion (BIC).

        Returns:
            The Bayesian Information Criterion value.
        """
        y_pred = self.pipeline.predict(self.X_val)
        n_params = len(self.pipeline.named_steps)
        n = len(self.y_val)
        mse = mean_squared_error(self.y_val, y_pred)
        bic = -2 * np.log(mse) + n_params * np.log(n)
        return bic

    def std(self):
        """
        Calculates the standard deviation of the predicted values.

        Returns:
            The standard deviation value.
        """
        return self.y_pred.std()

    def mean(self):
        """
        Calculates the mean of the predicted values.

        Returns:
            The mean value.
        """
        return self.y_pred.mean()

    def predict(self):
        """
        Performs prediction on the validation set.
        """
        self.y_pred = self.pipeline.predict(self.X_val)

    def set_frame_style(self, df, caption=""):
        """
        Helper function to set dataframe presentation style.

        Args:
            df: The DataFrame to style.
            caption: The caption for the styled DataFrame. Default is an empty string.

        Returns:
            The styled DataFrame.
        """
        return df.style.background_gradient(
            cmap='coolwarm').set_caption(caption).set_table_styles([{
                'selector':
                'caption',
                'props': [('color', 'Blue'), ('font-size', '28px'),
                          ('font-weight', 'bold')]
            }])

    def run(self):
        """
        Runs the regression metrics calculation and printing.
        """
        try:
            self.predict()
            metrics = {
                "Test R-squared": self.r2_test(),
                "Val R-squared": self.r2_val(),
                "MAE": self.mean_absolute_error(),
                "MSE": self.mean_squared_error(),
                "RMSE": self.mean_root_mean_squared_error(),
                "MAPE": self.mean_absolute_percentage_error(),
                "AIC": self.aic(),
                "BIC": self.bic(),
                "Std Deviation": self.std(),
                "Mean": self.mean()
            }

            df_metrics = pd.DataFrame.from_dict(metrics,
                                                orient="index",
                                                columns=["Value"])
            df_metrics.index.name = "Metric"

            if self.style:
                display(self.set_frame_style(df_metrics))
            else:
                print(df_metrics)
        except Exception as e:
            print("An error occurred:", str(e))


# Example

## Libs

In [2]:
from sklearn.datasets import fetch_california_housing
# Split
from sklearn.model_selection import train_test_split
# Model
import xgboost as xgb

from sklearn.pipeline import Pipeline

## Config

In [3]:
class CFG:
    #Random seed
    SEED = 42

## train test validate split

In [4]:
X, y = fetch_california_housing(as_frame=True,return_X_y =True)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,
                                                    random_state=CFG.SEED,
                                                    shuffle = True,
                                                   )
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50,
                                                random_state=CFG.SEED,
                                                shuffle = True,
                                               )

## Shape

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_val.shape, y_val.shape

((16512, 8), (2064, 8), (16512,), (2064,), (2064, 8), (2064,))

In [7]:
model = xgb.XGBRegressor()

In [8]:
steps = [('model', model)]  # Create a list of tuples with the steps
pipeline = Pipeline(steps=steps)

## Modeling

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('model',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=0, gpu_id=-1,
                              grow_policy='depthwise', importance_type=None,
                              interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_threshold=64, max_cat_to_onehot=4,
                              max_delta_step=0, max_depth=6, max_leaves=0,
                              min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=0, num_parallel_tree=1, predictor='auto',
                              random_state=0, ...))])

## Metrics

In [10]:
metrics = RegressionMetrics(pipeline, X_test, y_test, X_val, y_val)
metrics.run()

                    Value
Metric                   
Test R-squared   0.840350
Val R-squared    0.816672
MAE              0.317786
MSE              0.238944
RMSE             0.488819
MAPE             0.182462
AIC             12.495451
BIC             10.495451
Std Deviation    1.058918
Mean             2.082170


# With style

In [11]:
metrics = RegressionMetrics(pipeline, X_test, y_test, X_val, y_val, style = True)
metrics.run()

,Value
Metric,
Test R-squared,0.840350
Val R-squared,0.816672
MAE,0.317786
MSE,0.238944
RMSE,0.488819
MAPE,0.182462
AIC,12.495451
BIC,10.495451
Std Deviation,1.058918
